# Let's do something with data from the Eurovision Song Contest

Because why not

In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import uuid
import datetime

In [8]:
# CONSTANTS

BASE_URL = 'https://eurovision.tv'
TODAY = datetime.datetime.now().strftime('%Y%m%d')

In [4]:
# preliminary test – are we getting data right away?
response = requests.get('https://eurovision.tv/event/lisbon-2018/grand-final'
                        '/participants').text

soup = BeautifulSoup(response, 'lxml')

table_rows = soup.select('.row__TableRow-an9049-0')

for row in table_rows:
  print(row.text)
  

R/OCountryArtistsongPtsPointsRankRanking
01UkraineMELOVINUnder The Ladder13017
02SpainAmaia y AlfredTu Canción6123
03SloveniaLea SirkHvala, ne!6422
04LithuaniaIeva ZasimauskaitėWhen We're Old18112
05AustriaCesár SampsonNobody But You3423
06EstoniaElina NechayevaLa Forza2458
07NorwayAlexander RybakThat's How You Write A Song14415
08PortugalCláudia PascoalO Jardim3926
09United KingdomSuRieStorm4824
10SerbiaSanja Ilić & BalkanikaNova Deca11319
11GermanyMichael SchulteYou Let Me Walk Alone3404
12AlbaniaEugent BushpepaMall18411
13FranceMadame MonsieurMercy17313
14Czech RepublicMikolas JosefLie To Me2816
15DenmarkRasmussenHigher Ground2269
16AustraliaJessica MauboyWe Got Love9920
17FinlandSaara AaltoMonsters4625
18BulgariaEQUINOXBones16614
19MoldovaDoReDoSMy Lucky Day20910
20SwedenBenjamin IngrossoDance You Off2747
21HungaryAWSViszlát Nyár9321
22IsraelNettaTOY5291
23The NetherlandsWaylonOutlaw In 'Em12118
24IrelandRyan O'ShaughnessyTogether13616
25CyprusEleni FoureiraFuego4362
26ItalyErmal M

In [5]:
# Cool, so this works, which is nice – since that means that the data is 
# actually prerendered on the server and not generated by javascript in the 
# browser – although the way the website behaves certainly suggests so.

## Data Model

This should be the fully realised data model:

![Data Model](super-extended-model.svg)

Depending on the time, some of these tables might not exist and/or are already part of the main `song`table. 

## Contests

In [9]:
# Get all the contests from the entry page
contests_response = requests.get('https://eurovision.tv/events').text
contests_soup = BeautifulSoup(contests_response, 'lxml')


In [10]:

contests_elements = contests_soup.select('.result__Wrapper-s9upcfm-1')

contests = []

for contest_element in contests_elements:
  name = ' '.join(contest_element.select_one(
    '.text__Text-x6y36f-0').stripped_strings)
  location = re.search(r'^(.*) \d{4}', name).group(1)
  year = re.search(r'\d{4}', name).group(0)
  
  contest = {
    'url': contest_element['href'],
    'name': name,
    # Let's get the image, too, just for kicks
    'img_src': contest_element.select_one('img')['src'],
    'year': year,
    'location': location,
    'id': uuid.uuid3(uuid.NAMESPACE_URL, BASE_URL + contest_element['href'])
  }
  
  contests.append(contest)
  
contests_df = pd.DataFrame(contests)



In [6]:
contests_df

,id,img_src,location,name,url,year
0,c7aae442-7c52-3a14-9bf5-2fb1880fbf58,https://apex.eurovision.tv/image/52219b044b79f...,Lisbon,Lisbon 2018,/event/lisbon-2018,2018
1,42e0141d-3b05-3776-ba39-ea69c28e8bdb,https://apex.eurovision.tv/image/7b0be5f440549...,Kyiv,Kyiv 2017,/event/kyiv-2017,2017
2,9b967659-2937-3ddc-97a8-7d4296ecdc2b,https://apex.eurovision.tv/image/9ef19ed95f0ce...,Stockholm,Stockholm 2016,/event/stockholm-2016,2016
3,dcd86fee-f06a-315a-abcf-372a8fba2e7f,https://apex.eurovision.tv/image/6b2a4289be78d...,Vienna,Vienna 2015,/event/vienna-2015,2015
4,65ff473b-91bc-335f-bed3-9052fc7cdeb5,https://apex.eurovision.tv/image/a7c9ca682b37e...,Copenhagen,Copenhagen 2014,/event/copenhagen-2014,2014
5,fc7505da-0220-34ba-82a1-3abc5e8b3e36,https://apex.eurovision.tv/image/acf1012f98180...,Malmö,Malmö 2013,/event/malmo-2013,2013
6,1e72efe5-71b9-3669-9b83-b0857549f15b,https://apex.eurovision.tv/image/de26c67acd021...,Baku,Baku 2012,/event/baku-2012,2012
7,888687e1-8da0-3d6c-ab5a-ded1f7e9ac21,https://apex.eurovision.tv/image/06cbdc0fccf0e...,Düsseldorf,Düsseldorf 2011,/event/dusseldorf-2011,2011
8,459089e7-dc8b-3af3-816c-e195d8a63533,https://apex.eurovision.tv/image/2ef4dd5de1f60...,Oslo,Oslo 2010,/event/oslo-2010,2010
9,fa0099a0-6a5e-3a2e-913f-5e19be93c801,https://apex.eurovision.tv/image/a1834353e3c7e...,Moscow,Moscow 2009,/event/moscow-2009,2009


In [11]:
# save to csv
contests_df.to_csv('project/contests-' + TODAY + '.csv', index=False)

## Finalists

In [1]:


# Challenges: at some point during the contest, the format changed from just 
# having a final to having two semi-finals and a grand final – unfortunately,
#  this also means that the URL structure changes there. This certainly does 
# not make me that happy.

from time import sleep
from random import randint

In [2]:
def get_song_details(song_url):
  full_url = BASE_URL + song_url

  # Let's be cautious and not pelt the server with too many requests at once
  sleep(randint(5, 10))
  response = requests.get(full_url)

  if response.status_code > 200:
    print('  Request for {} failed.'.format(song_url))
    print('  Status:', response.status_code, response.content)
    return {}
  
  song_soup = BeautifulSoup(response.text, 'lxml')

  metadata_element = song_soup.select_one('[class*="songDetails"] + div')
  info_box_elements = metadata_element.select('div[class*="column"]')

  # Order of infoboxes:
  # - title
  # - performed by
  # - written by
  # - composed by
  # - broadcaster

  song_details = {
    'writers': info_box_elements[2].select_one('dd').text,
    'composers': info_box_elements[3].select_one('dd').text,
    'broadcaster': info_box_elements[4].select_one('dd').text
  }

  return song_details


In [38]:
get_song_details('/participant/imri')

{'broadcaster': 'IBA',
 'composers': 'Dolev Ram, Penn Hazut',
 'writers': 'Dolev Ram, Penn Hazut'}

In [3]:
def get_finalists(contest_url_fragement):
  full_url = BASE_URL + contest_url_fragement + '/final'

  response = requests.get(full_url)
  final_type = 'final'

  # Whoops, nothing to see at "/final". Let's try "/grand-final" instead
  if 200 < response.status_code < 500:
    full_url = BASE_URL + contest_url_fragement + '/grand-final'
    response = requests.get(full_url)
    final_type = 'grand final'

  contest_soup = BeautifulSoup(response.text, 'lxml')

  header_cells = contest_soup.select('table thead tr th '
                                     '.small-caps__SmallCaps-s1ooca2g-0')
  headers = [cell.text for cell in header_cells]

  finalists_rows = contest_soup.select('table tbody tr')

  finalists = []

  for finalist in finalists_rows:
    
    # get all cells in the row
    finalists_cells = finalist.select('td')
    
    # get some special cells, like the entry URL (which, strangely enough, 
    # is connected with the artist, and not the song
    song_url = finalists_cells[2].select_one('a')['href']
    
    # Set up the song entry
    entry = {
      'contest_id': uuid.uuid3(uuid.NAMESPACE_URL, BASE_URL + 
                               contest_url_fragement),
      'final_type': final_type,
      'url': song_url,
      'id': uuid.uuid3(uuid.NAMESPACE_URL, BASE_URL + song_url)
    }
    
    # go through the table row and add data to the dictionary
    for i in range(len(finalist)):
      entry[headers[i]] = finalists_cells[i].text
    
    # Find more data on the individual song page
    entry.update(get_song_details(song_url))
    
    #TODO Connect to last.fm API and look for even more data, because why not

    finalists.append(entry)

  return finalists


In [31]:
get_finalists('/event/bergen-1986')

5


5


5


5


5


...

5


5


5


5


[{'Artist': 'Sherisse Laurence',
  'Country': 'Luxembourg',
  'PtsPoints': '117',
  'R/O': '01',
  'RankRanking': '3',
  'broadcaster': 'CLT',
  'composers': 'Rolf Soja',
  'contest_id': UUID('0ee2efac-f244-3b1d-894f-7d1ebfb873ec'),
  'final_type': 'final',
  'id': UUID('51d675a7-dd90-3df7-91f0-f504cf032b54'),
  'song': "L'amour De Ma Vie",
  'url': '/participant/sherisse-laurence',
  'writers': 'Alain Garcia, Frank Dostal'},
 {'Artist': 'Doris Dragovic',
  'Country': 'Yugoslavia',
  'PtsPoints': '49',
  'R/O': '02',
  'RankRanking': '11',
  'broadcaster': 'JRT',
  'composers': 'Zrinko Tutic',
  'contest_id': UUID('0ee2efac-f244-3b1d-894f-7d1ebfb873ec'),
  'final_type': 'final',
  'id': UUID('b022a4bb-3398-3a44-940d-417edcb1c428'),
  'song': 'Zeljo Moja',
  'url': '/participant/doris-dragovic',
  'writers': '-'},
 {'Artist': 'Cocktail Chic',
  'Country': 'France',
  'PtsPoints': '13',
  'R/O': '03',
  'RankRanking': '17',
  'broadcaster': 'A2F',
  'composers': 'Georges Costa, Michel Co

In [12]:
# Okay, I guess we're just going to loop over the url fragments and use that 
# to create a new dataframe with all the contest entries that reached the final

finalists_list = []

counter = 0
for url in contests_df['url']:
  print('Get contest at', url)
  print('Index:', counter)
  sleep(randint(1,10))
  finalists_list = finalists_list + get_finalists(url)
  counter = counter + 1


In [16]:

finalists_df = pd.DataFrame(finalists_list)


In [17]:
finalists_df.to_csv('project/finalists-'+TODAY+'.csv', index=False)

In [41]:
finalists_list

[{'Artist': 'MELOVIN',
  'Country': 'Ukraine',
  'PtsPoints': '130',
  'R/O': '01',
  'RankRanking': '17',
  'broadcaster': 'UAPBC',
  'composers': 'MELOVIN',
  'contest_id': UUID('c7aae442-7c52-3a14-9bf5-2fb1880fbf58'),
  'final_type': 'grand final',
  'id': UUID('459bf9d8-4498-3fbe-9e2f-951162e4315b'),
  'song': 'Under The Ladder',
  'url': '/participant/melovin',
  'writers': 'Mike Ryals'},
 {'Artist': 'Amaia y Alfred',
  'Country': 'Spain',
  'PtsPoints': '61',
  'R/O': '02',
  'RankRanking': '23',
  'broadcaster': 'TVE',
  'composers': 'Raul Gomez Garcia, Sylvia Ruth Santoro Lopez',
  'contest_id': UUID('c7aae442-7c52-3a14-9bf5-2fb1880fbf58'),
  'final_type': 'grand final',
  'id': UUID('9c67c117-97fa-3f89-8fec-11d2d608f684'),
  'song': 'Tu Canción',
  'url': '/participant/alfred-amaia',
  'writers': 'Raul Gomez Garcia, Sylvia Ruth Santoro Lopez'},
 {'Artist': 'Lea Sirk',
  'Country': 'Slovenia',
  'PtsPoints': '64',
  'R/O': '03',
  'RankRanking': '22',
  'broadcaster': 'RTVSLO',